In [9]:
pip install pytorch-pretrained-bert


     -------------------------------------- 123.8/123.8 kB 7.1 MB/s eta 0:00:00
     ------------------------------------- 162.2/162.2 MB 13.1 MB/s eta 0:00:00
     -------------------------------------- 132.5/132.5 kB 8.2 MB/s eta 0:00:00
     ---------------------------------------- 79.6/79.6 kB ? eta 0:00:00
     ---------------------------------------- 9.1/9.1 MB 25.3 MB/s eta 0:00:00


In [6]:
pip install tensorflow

     -------------------------------------- 444.2/444.2 MB 3.1 MB/s eta 0:00:00
     ---------------------------------------- 123.4/123.4 kB ? eta 0:00:00
     --------------------------------------- 14.2/14.2 MB 27.3 MB/s eta 0:00:00
     ---------------------------------------- 57.5/57.5 kB ? eta 0:00:00
     ---------------------------------------- 42.6/42.6 kB 2.0 MB/s eta 0:00:00
     ------------------------------------- 438.7/438.7 kB 13.8 MB/s eta 0:00:00
     ------------------------------------- 895.5/895.5 kB 28.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 1.6/1.6 MB 25.7 MB/s eta 0:00:00
     ---------------------------------------- 65.5/65.5 kB ? eta 0:00:00
     ---------------------------------------- 3.6/3.6 MB 28.5 MB/s eta 0:00:00
     ---------------------------------------- 2.6/2.6 MB 23.7 MB/s eta 0:00:00
     -----------------------------------

In [3]:
pip install pytorch

  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [6 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "C:\Users\sksoh\AppData\Local\Temp\pip-install-_8wpytyd\pytorch_220a66c834e741eeba82254691ebc111\setup.py", line 15, in <module>
          raise Exception(message)
      Exception: You tried to install "pytorch". The package named for PyTorch is "torch"
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pytorch
  error: subprocess-exited-with-error
  
  × Running setup.py install for pytorch did not run successfully.
  │ exit code: 1
  ╰─> [6 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in 


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for pytorch
Failed to build pytorch
  Running setup.py install for pytorch: started
  Running setup.py install for pytorch: finished with status 'error'


In [3]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [4]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


In [6]:
import tensorflow as tf
import pandas as pd

df = pd.read_csv("./lyrics_polarity_dataset.csv")
df

,artist,title,year,lyrics,neg_sentiment,neu_sentiment,pos_sentiment,com_sentiment,polarity
0,frankie laine,i believe,1950,believe believe every drop rain fall flower gr...,0.158,0.785,0.056,-0.7506,0.800000
1,johnnie ray,cry,1950,cry sweetheart sends letter goodbye secret you...,0.184,0.612,0.204,0.2023,-0.164935
2,p?rez prado,patricia,1950,patricia lyricskiss lip always want patricia s...,0.028,0.583,0.388,0.9880,0.220000
3,lefty frizzell,if you've got the money i've got the time,1950,youve got money ive got time youve got money i...,0.041,0.721,0.237,0.9790,0.048413
4,lefty frizzell,i want to be with you always,1950,want always lose blue honey im one else youre ...,0.172,0.457,0.371,0.9604,0.080769
...,...,...,...,...,...,...,...,...,...
19534,nipsey hussle,hussle in the house,2019,hussle house look im comin straight slauson cr...,0.342,0.589,0.068,-0.9776,-0.126852
19535,the roots,the seed (2.0),2019,seed knocked month ago finna dont know want ne...,0.114,0.751,0.135,0.9049,-0.070603
19536,mack 10,10 million ways,2019,million way cause fuck u leave scar tick tock ...,0.203,0.680,0.117,-0.9684,0.012684
19537,nine,whutcha want?,2019,whutcha want get banned get banned dont someti...,0.171,0.662,0.168,-0.3603,-0.061637


In [7]:
def get_sent(pol):
    if (float(pol) > 0):
        return 1 # pos
    return 0 # neg + neu

for i in df.index.tolist():
  df.loc[i, "binary_pol"] = get_sent(df.loc[i, 'polarity'])

print(df.keys())

Index(['artist', 'title', 'year', 'lyrics', 'neg_sentiment', 'neu_sentiment',
       'pos_sentiment', 'com_sentiment', 'polarity', 'binary_pol'],
      dtype='object')


In [8]:
df.loc[0:5, "binary_pol"]

0    1.0
1    0.0
2    1.0
3    1.0
4    1.0
5    1.0
Name: binary_pol, dtype: float64

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['lyrics'], df['binary_pol'], test_size = 0.25, random_state = 32)

print(X_train.head())
print(y_train.head())

6139     introduction hey everybody please dont romp ro...
6903     mash youth man mash lyricsthis dish song youth...
16835    one lyricsall shall done say nothing new give ...
9299     eon got wishing well within thats ready begin ...
8885     blanket blue lyricsunder blanket blue beneath ...
Name: lyrics, dtype: object
6139     0.0
6903     0.0
16835    1.0
9299     0.0
8885     1.0
Name: binary_pol, dtype: float64


In [10]:
print(X_train.shape)
print(y_train.shape)


X_train_np = X_train.to_numpy()
X_test_np = X_test.to_numpy()
y_train_np = y_train.to_numpy()
y_test_np = y_test.to_numpy()


train = pd.DataFrame({'DATA_COL' : X_train_np, 'LABEL_COL' : y_train_np})
test = pd.DataFrame({'DATA_COL' : X_test_np, 'LABEL_COL' : y_test_np})

# train
test
test.shape

(14654,)
(14654,)


(4885, 2)

In [11]:
# Model
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model.summary()

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_75 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


In [20]:
def convert_data_to_examples(train, test, DATA_COL, LABEL_COL):
    train_InputExamples = train.apply(lambda x: InputExample(guid=None, text_a = x[DATA_COL], text_b = None, label = x[LABEL_COL]), axis = 1)
    validation_InputExamples = test.apply(lambda x: InputExample(guid = None, text_a = x[DATA_COL], text_b = None, label = x[LABEL_COL]), axis = 1)
    return train_InputExamples, validation_InputExamples


def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"], input_dict["token_type_ids"], input_dict['attention_mask'])
        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )

DATA_COL = 'DATA_COL'
LABEL_COL = 'LABEL_COL'

In [21]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(train, test, DATA_COL, LABEL_COL)

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\tokenization_utils_base.py:2323: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [22]:
train_InputExamples

0        InputExample(guid=None, text_a='introduction h...
1        InputExample(guid=None, text_a='mash youth man...
2        InputExample(guid=None, text_a='one lyricsall ...
3        InputExample(guid=None, text_a='eon got wishin...
4        InputExample(guid=None, text_a='blanket blue l...
                               ...                        
14649    InputExample(guid=None, text_a='visitor hear d...
14650    InputExample(guid=None, text_a='youre tuning f...
14651    InputExample(guid=None, text_a='new millennium...
14652    InputExample(guid=None, text_a='goddamn electr...
14653    InputExample(guid=None, text_a='girl next door...
Length: 14654, dtype: object

In [23]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model.fit(train_data, epochs=2, validation_data=validation_data)

Epoch 1/2
      7/Unknown - 90s 11s/step - loss: 0.6726 - accuracy: 0.6027

KeyboardInterrupt: 